In [ ]:
%%capture
%pip install llama-iris

In [15]:
%%capture
%pip install testcontainers-iris

In [1]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, StorageContext, ServiceContext
from llama_index.indices.vector_store import VectorStoreIndex
import openai

from llama_iris import IRISVectorStore

In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [6]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-02-10 11:32:04--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2024-02-10 11:32:04 (4.47 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [3]:
documents = SimpleDirectoryReader("./data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: ffb17cc2-4e38-4482-a96e-7ce339809cbf


In [ ]:
!docker pull containers.intersystems.com/intersystems/iris-community:2024.1-preview

In [1]:
import time
import os
from testcontainers.iris import IRISContainer

# Enterprise version with iris.key placed in the user's home directory
# license_key = os.path.abspath(os.path.expanduser("~/iris.key"))
# image = 'containers.intersystems.com/intersystems/iris:2023.3'

# Community Edition
license_key = None
image = 'containers.intersystems.com/intersystems/iris-community:2024.1-preview'

container = IRISContainer(image, username="demo", password="demo", namespace="demo", license_key=license_key)
container.with_exposed_ports(1972, 52773)
container.start()
CONNECTION_STRING = container.get_connection_url("localhost")

time.sleep(1)
print('Started', CONNECTION_STRING)
print('Portal: ', f"http://localhost:{container.get_exposed_port(52773)}/csp/sys/UtilHome.csp")


Pulling image containers.intersystems.com/intersystems/iris-community:2024.1-preview


ImageNotFound: 404 Client Error for http+docker://localhost/v1.44/images/containers.intersystems.com/intersystems/iris-community:2024.1-preview/json: Not Found ("No such image: containers.intersystems.com/intersystems/iris-community:2024.1-preview")

In [11]:
vector_store = IRISVectorStore.from_params(
    connection_string=CONNECTION_STRING,
    table_name="paul_graham_essay",
    embed_dim=1536,  # openai embedding dimension
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, llm=None
# )

index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
    # service_context=service_context,
)
query_engine = index.as_query_engine()

Generating embeddings: 100%|██████████| 22/22 [00:01<00:00, 15.59it/s]


In [12]:
response = query_engine.query("What did the author do?")

In [13]:
import textwrap
print(textwrap.fill(str(response), 100))

The author wrote a bunch of essays on various topics and also started thinking about other things
they could work on. They wrote an essay for themselves to answer the question of how they had chosen
what to work on in the past. They were surprised by the length and messiness of the answer, and
decided to write a more detailed version for others to read.


In [14]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))

In the mid 1980s, AI (Artificial Intelligence) was in the air and gaining attention. This was
influenced by a novel called "The Moon is a Harsh Mistress" by Heinlein, which featured an
intelligent computer called Mike, and a PBS documentary that showcased Terry Winograd using SHRDLU.
These events contributed to the interest and excitement surrounding AI during that time.
